<a href="https://colab.research.google.com/github/JanaStefan/IANNwTF_g22/blob/main/cnn_jana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import keras
from keras import layers, losses, optimizers
import numpy as np

In [3]:
def prep_cifar(train_data, test_data, batch_size, shuffle_buffer_size):
  def preprocessing_func(img, label):
    img = tf.cast(img, tf.float32)
    img = (img/128) - 1 #// normalized
    label = tf.one_hot(label, depth=10)
    return img, label
  train_data  = train_data.map(lambda img, label: preprocessing_func(img, label))
  train_data = train_data.shuffle(shuffle_buffer_size).batch(batch_size).prefetch(2)
  test_data = test_data.map(lambda img, label: preprocessing_func(img, label))
  test_data = test_data.batch(1)
  return train_data, test_data

In [43]:
class CNN():
    def __init__(self, input_size=(32, 32, 3), num_filters=[32, 64, 128], kernel_size=[(3,3), (3,3), (3,3)], strides=[(1,1), (1,1), (1,1)], conv_activation=['relu', 'relu', 'relu'], cnn_pool_type=["max_pool", "max_pool", "max_pool"],
                 padding=["valid", "valid", "valid"], use_bias=[False, False, False], dense_activation=['relu'], dense_sizes=[256], num_classes=10, flatten_type="global_max", name="CNN_model"):

        self.input_size = input_size
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.conv_activation = conv_activation
        self.cnn_pool_type = cnn_pool_type
        self.padding = padding
        self.use_bias = use_bias
        self.dense_activation = dense_activation
        self.dense_sizes = dense_sizes
        self.num_classes = num_classes
        self.flatten_type = flatten_type
        self.name = name

        self.model = None
        self.create_cnn()
        #self.compile_cnn()



    def create_cnn(self):
        inputs = layers.Input(shape=(self.input_size), dtype=tf.float32)
        x = inputs

        # Create CNN part with pooling layers
        for i, num_filter in enumerate(self.num_filters):
            x = layers.Conv2D(num_filter, self.kernel_size[i], self.strides[i], self.padding[i], activation=self.conv_activation[i], use_bias=self.use_bias[i])(x)
            x = layers.BatchNormalization()(x)
            if self.cnn_pool_type == "max_pool":
                x = layers.MaxPool2D()(x) #TODO: make the pool size adjustable too?
            else:
                x = layers.AveragePooling2D()(x) #TODO: make the pool size adjustable too?
            #x = layers.Dropout(0.2)(x)

        # Flatten the output of the CNN part for it to fit into the MLP part
        if self.flatten_type == "global_max":
            x = layers.GlobalMaxPool2D()(x)
        else:
            x = layers.GlobalAveragePooling2D()(x)

        # Create the MLP part
        for i, dense_size in enumerate(self.dense_sizes):
            x = layers.Dense(dense_size, activation=self.dense_activation[i])(x)

        # Create the output part
        y = layers.Dense(units=self.num_classes, activation='softmax')(x)

        self.model = keras.Model(inputs=inputs, outputs=y, name=self.name)
        self.model.summary()

    # not used:
    def compile_cnn(self):
        self.model.compile(loss=losses.SparseCategoricalCrossentropy(), optimizer=optimizers.Adam(), metrics=["accuracy"])


    def train(self, train_data, test_data, batch_size=64, shuffle_buffer_size=1000, epochs=15, learning_rate=0.01, optimizer='adam'):
        train_data, test_data = prep_cifar(train_data, test_data, batch_size, shuffle_buffer_size)
        if optimizer == 'adam':
          optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
        else:
          optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        loss_f = tf.keras.losses.CategoricalCrossentropy()
        for epoch in range(epochs):
          loss_list = []
          correct_predictions = 0
          total_samples = 0
          print("Epoch: " + str(epoch))
          for image, label in train_data:
            with tf.GradientTape() as tape:
              pred = self.model(image)
              loss = loss_f(label, pred)
              loss_list.append(loss)
            correct_predictions += np.sum(np.argmax(label, axis=1) == np.argmax(pred, axis=1))
            total_samples += len(train_data)
            gradients = tape.gradient(loss, self.model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
          print("Loss: " + str(np.average(loss_list)))
          print("Accuracy: " + str(correct_predictions / total_samples))
          print("correct pred: " + str(correct_predictions))
          print("num data: " + str(total_samples))

        test_loss, test_accuracy = self.evaluate(test_data)
        return test_loss, test_accuracy


    def evaluate(self, test_data):
      loss_f = tf.keras.losses.CategoricalCrossentropy()
      loss_list = []
      correct_predictions = 0
      for image, label in test_data:
          pred = self.model(image)
          loss = loss_f(label, pred)
          loss_list.append(loss)
          if np.argmax(label, axis=-1) == np.argmax(pred, axis=-1):
              correct_predictions += 1
      return np.average(loss_list), correct_predictions/len(test_data)


    def get_model(self):
        return self.model



In [29]:
import tensorflow_datasets as tfds
from keras.datasets import cifar10

train_data = tfds.load('cifar10', split='train', as_supervised=True)
test_data = tfds.load('cifar10', split='test', as_supervised=True)
#test_data = prep_cifar(test_data, 1, 1000)

In [44]:
# Default Set

cnn_model = CNN()
av_loss = cnn_model.train(train_data, test_data, epochs=1)
#av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

Model: "CNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_51 (Conv2D)          (None, 30, 30, 32)        864       
                                                                 
 batch_normalization_51 (Ba  (None, 30, 30, 32)        128       
 tchNormalization)                                               
                                                                 
 average_pooling2d_51 (Aver  (None, 15, 15, 32)        0         
 agePooling2D)                                                   
                                                                 
 conv2d_52 (Conv2D)          (None, 13, 13, 64)        18432     
                                                                 
 batch_normalization_52 (Ba  (None, 13, 13, 64)        25

Hyperparameter Tuning

In [ ]:
# Set 1:
cnn_model = CNN(
    num_filters=[32, 64, 128],
    kernel_size=[(3,3), (3,3), (3,3)],
    strides=[(1,1), (1,1), (1,1)],
    conv_activation=['elu', 'elu', 'elu'],
    cnn_pool_type=['max_pool', 'max_pool', 'max_pool'],
    padding=['valid', 'valid', 'valid'],
    use_bias=[False, False, False],
    dense_activation=['softmax'],
    dense_sizes=[128],
    flatten_type='global_max',
    name="Set_1",
)
cnn_model.train(train_data, test_data, batch_size=32, shuffle_buffer_size=500, epochs=15, learning_rate=0.0001, optimizer='rmsprop')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

Model: "Set_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_19 (Conv2D)          (None, 30, 30, 32)        864       
                                                                 
 batch_normalization_18 (Ba  (None, 30, 30, 32)        128       
 tchNormalization)                                               
                                                                 
 average_pooling2d_18 (Aver  (None, 15, 15, 32)        0         
 agePooling2D)                                                   
                                                                 
 conv2d_20 (Conv2D)          (None, 13, 13, 64)        18432     
                                                                 
 batch_normalization_19 (Ba  (None, 13, 13, 64)        256   

Loss: 2.2718532
Epoch: 1
Loss: 2.1984518
Epoch: 2
Loss: 2.1519659
Epoch: 3
Loss: 2.1097505
Epoch: 4
Loss: 2.069687
Epoch: 5
Loss: 2.0316186
Epoch: 6
Loss: 1.9940733
Epoch: 7
Loss: 1.9570284
Epoch: 8
Loss: 1.9213948
Epoch: 9
Loss: 1.8863839
Epoch: 10
Loss: 1.8518654
Epoch: 11
Loss: 1.8191652
Epoch: 12
Loss: 1.7874949
Epoch: 13


In [1]:
# Set 2:
cnn_model = CNN(
    num_filters=[32, 64, 128],
    kernel_size=[(4,4), (3,3), (2,2)],
    strides=[(2,2), (1,1), (1,1)],
    conv_activation=['elu', 'elu', 'elu'],
    cnn_pool_type=['max_pool', 'max_pool', 'max_pool'],
    padding=['valid', 'valid', 'valid'],
    use_bias=[False, False, False],
    dense_activation=['softmax'],
    dense_sizes=[128],
    flatten_type='global_max',
    name="Set_2",
)
cnn_model.train(train_data, test_data, batch_size=32, shuffle_buffer_size=1000, epochs=15, learning_rate=0.01, optimizer='rmsprop')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

NameError: ignored

In [ ]:
# Set 3:
cnn_model = CNN(
    num_filters=[64, 128, 128],
    kernel_size=[(4,4), (3,3), (2,2)],
    strides=[(2,2), (1,1), (1,1)],
    conv_activation=['relu', 'relu', 'relu'],
    cnn_pool_type=['max_pool', 'max_pool', 'max_pool'],
    padding=['valid', 'valid', 'valid'],
    use_bias=[False, True, True],
    dense_activation=['softmax'],
    dense_sizes=[128],
    flatten_type='global_max',
    name="Set_3",
)
cnn_model.train(train_data, test_data, batch_size=32, shuffle_buffer_size=1000, epochs=15, learning_rate=0.0001, optimizer='adam')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

In [ ]:
# Set 4:
cnn_model = CNN(
    num_filters=[64, 128, 128],
    kernel_size=[(4,4), (3,3), (2,2)],
    strides=[(2,2), (1,1), (1,1)],
    conv_activation=['relu', 'relu', 'relu'],
    cnn_pool_type=['max_pool', 'max_pool', 'max_pool'],
    padding=['valid', 'valid', 'valid'],
    use_bias=[False, True, True],
    dense_activation=['softmax'],
    dense_sizes=[128],
    flatten_type='global_max',
    name="Set_4",
)
cnn_model.train(train_data, test_data, batch_size=64, shuffle_buffer_size=500, epochs=15, learning_rate=0.01, optimizer='rmsprop')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

In [ ]:
# Set 5:
cnn_model = CNN(
    num_filters=[64, 64],
    kernel_size=[(3,3), (3,3)],
    strides=[(2,2), (2,2)],
    conv_activation=['relu', 'relu'],
    cnn_pool_type=['max_pool', 'max_pool'],
    padding=['valid', 'valid'],
    use_bias=[False, True],
    dense_activation=['softmax'],
    dense_sizes=[128],
    flatten_type='global_max',
    name="Set_5",
)
cnn_model.train(train_data, test_data, batch_size=64, shuffle_buffer_size=500, epochs=15, learning_rate=0.01, optimizer='rmsprop')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

In [ ]:
# Set 6:
cnn_model = CNN(
    num_filters=[64, 128],
    kernel_size=[(3,3), (3,3)],
    strides=[(2,2), (2,2)],
    conv_activation=['relu', 'relu'],
    cnn_pool_type=['av_pool', 'av_pool'],
    padding=['valid', 'valid'],
    use_bias=[False, True],
    dense_activation=['softmax'],
    dense_sizes=[128],
    flatten_type='av_max',
    name="Set_6",
)
cnn_model.train(train_data, test_data, batch_size=64, shuffle_buffer_size=500, epochs=15, learning_rate=0.01, optimizer='rmsprop')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

In [ ]:
# Set 7:
cnn_model = CNN(
    num_filters=[64, 128],
    kernel_size=[(3,3), (3,3)],
    strides=[(1,1), (1,1)],
    conv_activation=['relu', 'relu'],
    cnn_pool_type=['av_pool', 'av_pool'],
    padding=['valid', 'valid'],
    use_bias=[False, False],
    dense_activation=['tanh'],
    dense_sizes=[128],
    flatten_type='av_max',
    name="Set_7",
)
cnn_model.train(train_data, test_data, batch_size=32, shuffle_buffer_size=1000, epochs=15, learning_rate=0.01, optimizer='rmsprop')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))

In [ ]:
# Set 8:
cnn_model = CNN(
    num_filters=[32, 64, 128, 128],
    kernel_size=[(4,4), (3,3), (2,2), (2,2)],
    strides=[(2,2), (1,1), (1,1), (1,1)],
    conv_activation=['elu', 'elu', 'elu', 'elu'],
    cnn_pool_type=['max_pool', 'max_pool', 'max_pool', 'max_pool'],
    padding=['valid', 'valid', 'valid', 'valid'],
    use_bias=[False, False, False, False],
    dense_activation=['softmax'],
    dense_sizes=[128],
    flatten_type='global_max',
    name="Set_8",
)
cnn_model.train(train_data, test_data, batch_size=32, shuffle_buffer_size=1000, epochs=15, learning_rate=0.01, optimizer='rmsprop')
av_loss = cnn_model.evaluate(test_data)
print("Test Loss: " + str(av_loss))